In [1]:
import datetime
import logging
import os
import numpy as np
from ocean_navigation_simulator.reinforcement_learning.runners.GenerationRunner import (
    GenerationRunner,
)
import matplotlib.pyplot as plt
## Only when developing with VSCode in my repo 
#os.chdir('/home/nicolas/codeRepo/OceanPlatformControl') # here in Azure
# os.chdir('/home/nicolas/documents/Master_Thesis_repo/OceanPlatformControl') # here in WSL
# print(os.getcwd())
##
import pandas as pd
from ocean_navigation_simulator.utils.units import Distance
# These lines in VSCode to avoid reloading the kernel when changes are made to the external kernel
%load_ext autoreload
%autoreload 2

In [2]:
wandb_log_name = "FC_HC_GOM_30_platf_coll_100m_flocking_0_5"
wandb_log = False

### Define Filenames

+ Hindcast-Hindcast

In [3]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = '30_platforms_HC_HC_col_100m_hj_naive_1181.csv'
pb_name_reactive = '30_platforms_HC_HC_col_100m_reactive_ctrl_1174.csv'
pb_name_flocking = '30_platforms_HC_HC_col_100m_flocking_1_1179.csv' #'30_platforms_HC_HC_col_100m_flocking_0_5_1120.csv'#'30_platforms_HC_HC_col_100m_flocking_0_1_1193.csv' #'30_platforms_HC_HC_flocking_0_1_1179.csv'

+ Forecast-Hindcast

In [4]:
path_to_files = 'generated_media/metrics_csv_results/'
#file_containing_pb = 'problemsGOM.csv'
pb_name_hj_naive = '30_platforms_FC_HC_col_100m_hj_naive_1122_fiedler.csv'
pb_name_reactive = '30_platforms_FC_HC_col_100m_reactive_ctrl_1126_fiedler.csv'
pb_name_flocking = '30_platforms_FC_HC_col_100m_flocking_0_5_1121_fiedler.csv'

### Plot problem distribution

In [ ]:
# TODO

### Extract metrics from csv files

In [5]:
metrics_hj_naive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_hj_naive}")
metrics_hj_reactive_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_reactive}")
metrics_hj_flocking_unfiltered = pd.read_csv(f"{path_to_files}/{pb_name_flocking}")

+ Make sure we have the (same) indices and total number of missions for the panda dataframes

In [6]:
# Find the common mission_nr values
common_mission_nr = set(metrics_hj_naive_unfiltered['mission_nr']).intersection(set(metrics_hj_reactive_unfiltered['mission_nr'])).intersection(set(metrics_hj_flocking_unfiltered['mission_nr']))
# Filter the rows in each dataframe that have matching mission_nr values
metrics_hj_naive = metrics_hj_naive_unfiltered[metrics_hj_naive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_reactive = metrics_hj_reactive_unfiltered[metrics_hj_reactive_unfiltered['mission_nr'].isin(common_mission_nr)]
metrics_hj_flocking = metrics_hj_flocking_unfiltered[metrics_hj_flocking_unfiltered['mission_nr'].isin(common_mission_nr)]

+ Sort by ascending missions nr order

In [7]:
metrics_hj_naive = metrics_hj_naive.sort_values(['mission_nr'])
metrics_hj_reactive= metrics_hj_reactive.sort_values(['mission_nr'])
metrics_hj_flocking = metrics_hj_flocking.sort_values(['mission_nr'])

+ Index by mission nr to get the same order in the panda dataframe between controller metrics

In [8]:
metrics_hj_naive.set_index('mission_nr', inplace=True)
metrics_hj_reactive.set_index('mission_nr', inplace=True)
metrics_hj_flocking.set_index('mission_nr', inplace=True)

+ Get Non Feasible Missions 

In [9]:
non_feas_idx = metrics_hj_naive.index[metrics_hj_naive["Reaching_target"] < 1]
ratio_non_feas_miss = len(non_feas_idx)/len(metrics_hj_naive)*100
print("Number of infeasible missions: ", len(non_feas_idx), f"corresponds to {ratio_non_feas_miss} %")

Number of infeasible missions:  825 corresponds to 81.36094674556213 %


+ Drop infeasible missions on HC

In [ ]:
# metrics_hj_naive.drop(non_feas_idx, axis=0, inplace=True)
# metrics_hj_reactive.drop(non_feas_idx, axis=0, inplace=True)
# metrics_hj_flocking.drop(non_feas_idx, axis=0, inplace=True)

In [10]:
import plotly.express as px
import plotly.graph_objects as go

In [10]:
print("number of missions considered: ", len(metrics_hj_naive))

number of missions considered:  1014


In [36]:
metrics_hj_naive['fiedler_min'].mean()

0.25172532054203334

In [37]:
metrics_hj_reactive['fiedler_min'].mean()

0.34078148791595386

In [38]:
metrics_hj_flocking['fiedler_min'].mean()

1.2920596442406889

In [11]:
metrics_hj_naive[metrics_hj_naive['Isolated_platform_metric']>0]

,Unnamed: 0,Isolated_platform_metric,Number_of_collision,Reaching_target,Mean minimum distance to target among all platforms in deg,Mean maximum correction from optimal control degrees,Average Beta Index,Initial maximum degree of the graph,Final maximum degree of the graph,Mission_success,mean solver time in seconds,std solver time in seconds,fiedler_mean,fiedler_min,fiedler_max,nb_disconnections
mission_nr,,,,,,,,,,,,,,,,
0,0,1.278164,174,0.000000,0.341171,0.0,9.025048,11,28,0,3.255585,4.600506,2.694495,0.000000e+00,14.488458,76
1,1,0.060185,0,0.533333,0.028881,0.0,5.573025,9,26,0,3.169761,3.937709,3.443675,7.255827e-01,10.938707,0
2,295,1.096065,75,0.600000,0.126338,0.0,1.918189,9,9,0,3.370340,4.504446,0.097998,-1.688715e-15,1.259343,81
3,392,2.542824,0,0.466667,0.064277,0.0,2.055183,15,7,0,3.096417,4.157399,0.507247,-1.553407e-15,4.321330,58
5,586,1.030478,0,0.333333,0.206447,0.0,2.145780,11,8,0,2.927992,3.576136,0.310609,-1.802919e-15,1.381522,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1190,205,2.375000,0,0.100000,0.230850,0.0,3.595645,10,14,0,3.420862,4.730390,0.181382,-2.223555e-15,3.705461,87
1191,206,2.806713,0,0.133333,0.250794,0.0,4.502158,10,17,0,3.432572,4.775898,0.020255,-2.925714e-15,0.848580,93
1192,207,2.173611,0,0.333333,0.041764,0.0,2.517341,13,7,0,3.211914,4.024302,0.197247,-2.864385e-15,1.651441,54


## Box Plots

In [19]:
from plotly.offline import  init_notebook_mode
init_notebook_mode(connected=True)

+ Mean Fiedler value 

In [40]:
fiedler_dict = {"hj_naive": metrics_hj_naive["fiedler_mean"],
                      "hj_reactive": metrics_hj_reactive["fiedler_mean"],
                      "flocking": metrics_hj_flocking["fiedler_mean"]}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all")
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Fiedler value over Missions"
)
fig_beta.show()

In [28]:
fiedler_dict = {"hj_naive": np.abs(metrics_hj_naive["fiedler_min"]),
                      "hj_reactive": np.abs(metrics_hj_reactive["fiedler_min"]),
                      "flocking": np.abs(metrics_hj_flocking["fiedler_min"])}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all")
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
#Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    height=800,
    width=1000
)
# fig_beta.update_layout(
#     xaxis_title="controllers",
#     yaxis_title="Minimum Fiedler value over Missions",
#     height=800,
#     width=1000,
#     yaxis=dict(
#         type='log',
#         #dtick=0.2,
#         tickvals=[1,2],
#         ticktext=['0','1' ],
#         range=[0,2]
#         # showexponent='none'
#     )
# )
fig_beta.show()

+ Min Fiedler value 

In [27]:
fiedler_dict = {"hj_naive": np.abs(metrics_hj_naive["fiedler_min"])+1,
                      "hj_reactive": np.abs(metrics_hj_reactive["fiedler_min"])+1,
                      "flocking": np.abs(metrics_hj_flocking["fiedler_min"])+1}
beta_index_avg_df = pd.DataFrame(fiedler_dict)
fig_beta = px.violin(beta_index_avg_df, points="all", log_y=True)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
# fig_beta.update_layout(
#     xaxis_title="controllers",
#     yaxis_title="Minimum Fiedler value over Missions",
#     height=800,
#     width=1000
# )
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    height=800,
    width=1000,
    yaxis=dict(
        type='log',
        #dtick=0.2,
        tickvals=[1,10*np.log10(2+1), 10*np.log10(5+1)],
        ticktext=['0','2', '5' ],
        range=[0,1]
        # showexponent='none'
    )
)
fig_beta.show()

In [167]:
np.log(min(fiedler_dict["hj_naive"])+1e-15)

-34.538776394910684

In [85]:
import plotly.express as px
import pandas as pd

# Create data frame
fiedler_dict = {"hj_naive": metrics_hj_naive["fiedler_min"],
                "hj_reactive": metrics_hj_reactive["fiedler_min"],
                "flocking": metrics_hj_flocking["fiedler_min"]}
beta_index_avg_df = pd.DataFrame(fiedler_dict)

# Create a list of colors based on the column values
colors = []
for col in beta_index_avg_df.columns:
    colors += ['red' if x < 1 else 'blue' for x in beta_index_avg_df[col]]
    # colors += ['red' if x < 1 else 'red' for x in beta_index_avg_df[col]]

# Create violin plot
fig_beta = px.box(beta_index_avg_df.melt(var_name='Controller', value_name='Minimum Fiedler value over Missions'), x='Controller', y='Minimum Fiedler value over Missions',
                      boxmode='overlay',color=colors, points="all")

# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="Controllers",
    yaxis_title="Minimum Fiedler value over Missions",
    legend_traceorder='reversed',
    height=800,
    width=1000
)
# Update the legend labels
fig_beta.update_traces(
    name='below 1',
    selector={'legendgroup': 'red'}
)
fig_beta.update_traces(
    name='above 1',
    selector={'legendgroup': 'blue'}
)

fig_beta.show()


In [45]:
beta_index_avg_df['hj_naive']

mission_nr
0       0.000000e+00
1       7.753889e-01
2      -1.632120e-15
3      -1.642506e-15
5      -8.272148e-16
            ...     
1193   -1.492569e-15
1196    1.140877e+00
1197    1.019299e-04
1198    1.547686e+00
1199    6.638187e-01
Name: hj_naive, Length: 1014, dtype: float64

+ Beta Index

In [13]:
beta_index_avg_dic = {"hj_naive": metrics_hj_naive["Average Beta Index"],
                      "hj_reactive": metrics_hj_reactive["Average Beta Index"],
                      "flocking": metrics_hj_flocking["Average Beta Index"]}
beta_index_avg_df = pd.DataFrame(beta_index_avg_dic)
fig_beta = px.violin(beta_index_avg_df)
mean_trace = go.Scatter(
    x=beta_index_avg_df.columns,
    y=beta_index_avg_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_beta.add_trace(mean_trace)
# Add custom x and y labels
fig_beta.update_layout(
    xaxis_title="controllers",
    yaxis_title="Average Beta Index over Missions"
)
fig_beta.show()

+ Mean minimum distance to target among all platforms km

In [ ]:
mean_min_dist_dict = {"hj_naive": Distance(deg=metrics_hj_naive["Mean minimum distance to target among all platforms in deg"]).km,
                      "hj_reactive": Distance(deg=metrics_hj_reactive["Mean minimum distance to target among all platforms in deg"]).km,
                      "flocking":Distance(deg=metrics_hj_flocking["Mean minimum distance to target among all platforms in deg"]).km}
idx_list = []
mean_min_dist_df = pd.DataFrame(mean_min_dist_dict)
fig_min_dist = px.violin(mean_min_dist_df, log_y=True)
mean_trace = go.Scatter(
    x=mean_min_dist_df.columns,
    y=mean_min_dist_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_min_dist.add_trace(mean_trace)
# Add custom x and y labels
fig_min_dist.update_layout(
    xaxis_title="controllers",
    yaxis_title=" [km]",
    title = {'text': "Mean minimum distance to target among all platforms"}
)
fig_min_dist.show()

+ Mean maximum correction from optimal control degrees

In [ ]:
opt_ctrl_correction_dict = {"hj_naive": metrics_hj_naive["Mean maximum correction from optimal control degrees"],
                      "hj_reactive": metrics_hj_reactive["Mean maximum correction from optimal control degrees"],
                      "flocking": metrics_hj_flocking["Mean maximum correction from optimal control degrees"]}
opt_ctrl_correction_df = pd.DataFrame(opt_ctrl_correction_dict)
fig_opt_ctrl_correction = px.box(opt_ctrl_correction_df )
# Add custom x and y labels
fig_opt_ctrl_correction.update_layout(
    xaxis_title="controllers",
    yaxis_title="degrees",
    title = {'text': "Control direction correction from optimal input"}
)
fig_opt_ctrl_correction.show()

+ Isolated Platform Metric

In [ ]:

failed_connectivity_hj_flocking =  metrics_hj_flocking#metrics_hj_flocking[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_naive = metrics_hj_naive#metrics_hj_naive[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_reactive = metrics_hj_reactive #metrics_hj_reactive[metrics_hj_flocking["Isolated_platform_metric"]>0]


ipm_dict= {"hj_naive": failed_connectivity_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_connectivity_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_connectivity_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"}
)
fig_ipm.show()

In [89]:
failed_connectivity_hj_flocking =  metrics_hj_flocking#metrics_hj_flocking[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_naive = metrics_hj_naive#metrics_hj_naive[metrics_hj_flocking["Isolated_platform_metric"]>0]
failed_connectivity_hj_reactive = metrics_hj_reactive #metrics_hj_reactive[metrics_hj_flocking["Isolated_platform_metric"]>0]


ipm_dict= {"hj_naive": failed_connectivity_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_connectivity_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_connectivity_hj_flocking["Isolated_platform_metric"]}
ipm_df = pd.DataFrame(ipm_dict)
fig_ipm = px.box(ipm_df, log_y=False )
mean_trace = go.Scatter(
    x=ipm_df.columns,
    y=ipm_df.mean(),
    mode='lines',
    line=dict(color='green', width=2),
    showlegend=True,
    name="mean"
)
fig_ipm.add_trace(mean_trace)
# Add custom x and y labels
fig_ipm.update_layout(
    xaxis_title="controllers",
    yaxis_title="IPM",
    title = {'text': "Isolated_platform_metric"},
    height=800,
    width=1000
)
fig_ipm.show()

In [90]:
# extract missions for which flocking failed to maintain connectivity
failed_idx_flocking = metrics_hj_flocking["Isolated_platform_metric"]>0

failed_hj_flocking =  metrics_hj_flocking[failed_idx_flocking]
failed_hj_naive = metrics_hj_naive[failed_idx_flocking]
failed_hj_reactive = metrics_hj_reactive[failed_idx_flocking]

ipm_dict_all= {"hj_naive": metrics_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": metrics_hj_reactive["Isolated_platform_metric"],
                      "flocking":metrics_hj_flocking["Isolated_platform_metric"]}

ipm_dict_failed_connect= {"hj_naive": failed_connectivity_hj_naive["Isolated_platform_metric"],
                      "hj_reactive": failed_connectivity_hj_reactive["Isolated_platform_metric"],
                      "flocking":failed_connectivity_hj_flocking["Isolated_platform_metric"]}

all_x_ticks = ['hj_naive']*len(metrics_hj_naive) + ['hj_reactive']*len(metrics_hj_reactive) + ['flocking']*len(metrics_hj_flocking)
all_missions = list(metrics_hj_naive["Isolated_platform_metric"])+list(metrics_hj_reactive["Isolated_platform_metric"])+ list(metrics_hj_flocking["Isolated_platform_metric"])
failed_x_ticks = ['hj_naive']*len(failed_hj_naive) + ['hj_reactive']*len(failed_hj_reactive) + ['flocking']*len(failed_hj_flocking)
failed_missions = list(failed_hj_naive["Isolated_platform_metric"])+list(failed_hj_reactive["Isolated_platform_metric"])+ list(failed_hj_flocking["Isolated_platform_metric"])
fig = go.Figure()
# HJ naive
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color='#636EFA'), name = 'all missions'))
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color='#EF553B'), name = 'failed missions only'))
fig.update_layout(
    yaxis_title='IPM',
    boxmode='group', # group together boxes of the different traces for each value of x
    height=800,
    width=1000
)

fig.show()

In [120]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1,horizontal_spacing= 0.075)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color=failed_missions_color), name='Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='IPM', row=1, col=1)
fig.update_yaxes(title_text='IPM', row=1, col=2)

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=800, width=1200)

# Show the plot
fig.show()

In [44]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

failed_idx_flocking = metrics_hj_flocking["nb_disconnections"]>0
failed_hj_flocking =  metrics_hj_flocking[failed_idx_flocking]
failed_hj_naive = metrics_hj_naive[failed_idx_flocking]
failed_hj_reactive = metrics_hj_reactive[failed_idx_flocking]

# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1,horizontal_spacing= 0.075)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

all_x_ticks = ['hj_naive']*len(metrics_hj_naive) + ['hj_reactive']*len(metrics_hj_reactive) + ['flocking']*len(metrics_hj_flocking)
all_missions = list(metrics_hj_naive["Isolated_platform_metric"])+list(metrics_hj_reactive["Isolated_platform_metric"])+ list(metrics_hj_flocking["Isolated_platform_metric"])
failed_x_ticks = ['hj_naive']*len(failed_hj_naive) + ['hj_reactive']*len(failed_hj_reactive) + ['flocking']*len(failed_hj_flocking)
failed_missions = list(failed_hj_naive["Isolated_platform_metric"]/failed_hj_naive['nb_disconnections'])+list(failed_hj_reactive["Isolated_platform_metric"]/failed_hj_naive['nb_disconnections'])+ \
                  list(failed_hj_flocking["Isolated_platform_metric"]/failed_hj_naive['nb_disconnections'])

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints= 'all', line=dict(color=failed_missions_color), name='Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='IPM', row=1, col=1)
fig.update_yaxes(title_text='Disconnected time in min', row=1, col=2, type='log')

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=800, width=1200)

# Show the plot
fig.show()

In [119]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the number of rows and columns for the subplots
nrows = 1
ncols = 2

# Create the subplots
fig = make_subplots(rows=nrows, cols=ncols, vertical_spacing=0.1, horizontal_spacing=0.075)

# Define the colors for the box plots
all_missions_color = '#636EFA'
failed_missions_color = '#EF553B'

# Add the box plots to the subplots
fig.add_trace(go.Box(x=all_x_ticks, y=all_missions, jitter=0.25, pointpos=-1.6, boxpoints='all',  line=dict(color=all_missions_color), name='All Missions'), row=1, col=1)
fig.add_trace(go.Box(x=failed_x_ticks, y=failed_missions, jitter=0.25, pointpos=+1.6, boxpoints='all', line=dict(color=failed_missions_color), name='Failed Missions Only'), row=1, col=2)

# Set the y-axis label for the subplots
fig.update_yaxes(title_text='IPM', row=1, col=1)
fig.update_yaxes(title_text='IPM', row=1, col=2)

# Set the subplot titles
fig.update_layout(title_text='Isolated Platform Metric by Mission Type', title_x=0.5)

# Set the box mode to "group" for both subplots
#fig.update_layout(boxmode='group')

# Set the dimensions of the figure
fig.update_layout(height=600, width=1200)

# Set the legend for each subplot
# Set the legend for each subplot
fig.update_layout(
    legend=dict(
        traceorder='normal',
        font=dict(size=12),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2,
        orientation='v',
        yanchor='top',
        xanchor='left',
        y=1.0,
        x=0.01
    ),
    legend_tracegroupgap=20,
    # Set the blue legend to only show the "All Missions" item
    legend_items=[dict(label='All Missions', color=all_missions_color)]
)

fig.update_layout(
    legend=dict(
        traceorder='normal',
        font=dict(size=12),
        bgcolor='LightSteelBlue',
        bordercolor='Black',
        borderwidth=2,
        orientation='v',
        yanchor='top',
        xanchor='left',
        y=1.0,
        x=0.55
    ),
    legend_tracegroupgap=20,
    # Set the red legend to only show the "Failed Missions Only" item
    legend_items=[dict(label='Failed Missions Only', color=failed_missions_color)]
)


# Show the plot
fig.show()


ValueError: Invalid property specified for object of type plotly.graph_objs.layout.Legend: 'items'

Did you mean "title"?

    Valid properties:
        bgcolor
            Sets the legend background color. Defaults to
            `layout.paper_bgcolor`.
        bordercolor
            Sets the color of the border enclosing the legend.
        borderwidth
            Sets the width (in px) of the border enclosing the
            legend.
        entrywidth
            Sets the width (in px or fraction) of the legend. Use 0
            to size the entry based on the text width, when
            `entrywidthmode` is set to "pixels".
        entrywidthmode
            Determines what entrywidth means.
        font
            Sets the font used to text the legend items.
        groupclick
            Determines the behavior on legend group item click.
            "toggleitem" toggles the visibility of the individual
            item clicked on the graph. "togglegroup" toggles the
          

### Get Metrics summary as a dict:

In [ ]:
def get_metrics_summary(metrics_file:pd.DataFrame, ctrl_name: str)->dict:
    return {
    "ctrl_name": ctrl_name,
    "avg_beta_index":metrics_file["Average Beta Index"].mean(),
    "avg_isolated_pltf_integral": metrics_file["Isolated_platform_metric"].mean(),
    "avg_correction_from_opt_ctrl_deg": metrics_file["Mean maximum correction from optimal control degrees"].mean(),
    "avg_mean_min_dist_to_target_km": Distance(deg=metrics_file["Mean minimum distance to target among all platforms in deg"].mean()).km,
    "avg_nb_platfroms_reaching_target": metrics_file["Reaching_target"].mean(),
    "nb_missions_with_isolated_pltf": np.sum(metrics_file["Isolated_platform_metric"] > 0),
    "rate_of_missions_with_disconnection":np.mean(metrics_file["Isolated_platform_metric"] > 0), 
    "nb_missions_with_collisions": np.sum(metrics_file["Number_of_collision"] > 0),
    "rate_of_missions_with_collisions": np.mean(metrics_file["Number_of_collision"] > 0),
    "nb_missions_success": np.sum(metrics_file["Mission_success"]==1),
}

  + HJ Multi-Agent Naive

In [ ]:
metrics_summary_hj_naive = get_metrics_summary(metrics_hj_naive, ctrl_name="hj_naive")
print(metrics_summary_hj_naive)
summary_hj_naive_df = pd.DataFrame(metrics_summary_hj_naive, index=[0])

+ HJ Multi-Agent Decentralized Reactive Control

In [ ]:
metrics_summary_hj_reactive = get_metrics_summary(metrics_hj_reactive, ctrl_name='hj_reactive')
print(metrics_summary_hj_reactive)
summary_hj_naive_reactive = pd.DataFrame(metrics_summary_hj_reactive, index=[0])

+ HJ Multi-Agent Flocking

In [ ]:
metrics_summary_hj_flocking = get_metrics_summary(metrics_hj_flocking, ctrl_name='hj_flocking')
print(metrics_summary_hj_flocking)
summary_hj_naive_flocking= pd.DataFrame(metrics_summary_hj_flocking, index=[0])

## Confusion Matrices Plots

In [ ]:
def plot_confusion_matrix(x, y, x_label, y_label, title_name:str):
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.colors import LogNorm

    confusion_matrix = pd.crosstab(x, y, rownames=[x_label], colnames=[y_label], normalize=True)*100
    fig = px.imshow(confusion_matrix, text_auto=True)
    fig.update_traces(texttemplate='%{text:.1f}%', textfont_size=12, text=confusion_matrix.values)
    #fig = px.imshow(confusion_matrix.round(2), text_auto=True)
    fig.update_layout(
        title = {'text': title_name}
    )
    fig.show()
    # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, fmt='g')
    # # ax = sns.heatmap(confusion_matrix, annot=True, cmap=plt.cm.RdYlGn, square=True, norm=LogNorm(), fmt='g') # if heat map is in norm
    # plt.tick_params(axis='both', which='major', labelbottom=False, bottom=False, top=True, labeltop=True)
    # ax.xaxis.set_label_position('top')
    # ax.plot
    # plt.title(title_name)
    # plt.show()
    return fig

###  HJ naive vs Decentralized Reactive Control

+  Missions with collisions

In [ ]:
fig_conf_react_coll = plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_reactive["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with collisions")

+ Missions with communication losses

In [ ]:
fig_conf_react_comm_loss= plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_reactive["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [ ]:
connect_objective_reactive = (metrics_hj_reactive["Isolated_platform_metric"]==0) & (metrics_hj_reactive["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_react_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_reactive , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Reactive Control', title_name= 'Missions without collisions and with maintained connectivity')

+  Missions success counts

In [ ]:
fig_react_success  = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_reactive["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Reactive', title_name="Mission Success")

###  HJ naive vs Flocking

+  Missions with collisions

In [ ]:
fig_conf_flocking_coll= plot_confusion_matrix(metrics_hj_naive["Number_of_collision"]>0, 
                        metrics_hj_flocking["Number_of_collision"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with collisions")

+ Missions with communication losses

In [ ]:
fig_conf_flocking_comm_loss = plot_confusion_matrix(metrics_hj_naive["Isolated_platform_metric"]>0, 
                        metrics_hj_flocking["Isolated_platform_metric"]>0, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking Control', title_name="Missions with communication losses")

+ Connectivity and Collision maintenance performance

In [ ]:
connect_objective_flocking = (metrics_hj_flocking["Isolated_platform_metric"]==0) & (metrics_hj_flocking["Number_of_collision"] == 0)
connect_objective_hj_naive = (metrics_hj_naive["Isolated_platform_metric"]==0) & (metrics_hj_naive["Number_of_collision"] == 0)
fig_flocking_maint = plot_confusion_matrix(connect_objective_hj_naive , connect_objective_flocking , 
    'Multi-Agent HJ Naive', 'Multi-Agent HJ Flocking Control', title_name= 'Missions without collisions and with maintained connectivity')

In [ ]:
index_list = metrics_hj_flocking.index[(connect_objective_reactive==True) & (connect_objective_flocking==False)].tolist()


In [ ]:
index_list

In [ ]:
metrics_hj_reactive.loc[285]

+  Missions success counts

In [ ]:
fig_flocking_success = plot_confusion_matrix(metrics_hj_naive["Mission_success"]==1, 
                        metrics_hj_flocking["Mission_success"]==1, 'Multi-Agent HJ Naive', 
                        'Multi-Agent HJ Flocking', title_name="Mission Success")

## Log Data in WandB

In [ ]:
import wandb

In [ ]:
# Log metrics in WandB
if wandb_log:
    os.environ["WANDB_API_KEY"] = "1f19232e6ccc9981a8a972bee18ba31a94644835"

    wandb.init(
        # Set the project where this run will be logged
        project="Master_Thesis_Run_Summaries",
        # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
        name=wandb_log_name,
        # Track hyperparameters and run metadata
        config={
            "Number of infeasible missions": len(non_feas_idx),
            "Percentage of infeasible missions": ratio_non_feas_miss,
            "Number of missions feasible": len(metrics_hj_naive),
            "Ratio failing missions with HJ naive in terms of connectivity and collisions": 
            len(metrics_hj_naive.index[(connect_objective_hj_naive==False)].tolist())/len(metrics_hj_naive),
        },
        entity="nhoischen",
    )
    wandb.log({"Metrics_Summary": wandb.Table(dataframe=pd.concat([summary_hj_naive_df, summary_hj_naive_reactive, summary_hj_naive_flocking]))},
                commit=False)
    wandb.log({'Beta Index': fig_beta,
            'Time-Integral-over-Isolated-Platforms': fig_ipm,
            'Platforms-Mean-Min-Distance-to-Target': fig_min_dist,
            'Deviation-from-Cptimal-Ctrl-Direction': fig_opt_ctrl_correction,
                },
            commit=False)
    wandb.log({'Missions with collisions hj-reactive': fig_conf_react_coll,
            'Missions with collisions hj-flocking': fig_conf_flocking_coll,
            'Missions with communication losses hj-reactive': fig_conf_react_comm_loss,
            'Missions with communication losses hj-flocking': fig_conf_flocking_comm_loss,
            'Missions with maintained connectivity and no collisions hj-reactive': fig_react_maint,
            'Missions with maintained connectivity and no collisions hj-flocking': fig_flocking_maint,
            'Succesfull Missions hj-reactive': fig_react_success,
            'Succesfull Missions hj-flocking': fig_flocking_success}, commit=True)
    wandb.finish()